<a href="https://colab.research.google.com/github/SooHwanIT/Alert/blob/master/%EA%B2%8C%EC%9E%84_%ED%8F%89%EA%B0%80_%EC%98%88%EC%B8%A1_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 임포트

In [ ]:
# 판다스 모듈 임포트
import pandas as pd

# 구글 드라이브 임포트
from google.colab import drive
drive.mount('/content/drive')

# 넘파이 일포트
import numpy as np

# 사이킷런 임포트
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_breast_cancer



Mounted at /content/drive


# 전처리

In [ ]:
# 파일 임포트
df = pd.read_csv('/content/drive/MyDrive/steam_games.csv',sep=',')

# 게임이 아닌 데이터 (패키지 등) 제거
df = df[df['types'] == 'app']

# 분류에 필요한 열만 가져오기
#1단계
df = df[['all_reviews','developer','game_details','languages','genre','original_price']]

#2단계
# df = df[['all_reviews','release_date','developer','game_details','languages','achievements','genre','mature_content','original_price']]

# 리뷰 데이터 퍼센트로 바꾸기
df["all_reviews"] = df["all_reviews"].str.extract(r'(\d+%)')
df["all_reviews"] = df["all_reviews"].astype('str').str.replace("%","")

original_price = df["all_reviews"].isin(["nan"])
df = df[~original_price]

# 가격 데이터 결손값 지우기, 무료=>0 으로 교체
df["original_price"]=df["original_price"].astype('str').str.replace("Free","$0")

df["original_price"]=df["original_price"].astype('str').str.replace("$","")

# 리뷰에 결손치가 존재하는 행 제거
df = df.dropna(axis=0)

# 전처리 후 데이터 프레임 생성
global n_df
n_df = pd.DataFrame()
n_df.insert(0,'all_reviews',df['all_reviews'])
# n_df.insert(0,'original_price',df['original_price'])

def MLB(label):
  global n_df

  # 문자여 배열로 바꾸기
  df[label] = df[label].str.split(',')

  # 멀티플 레이블 비널라이즈?
  mlb =  MultiLabelBinarizer()
  labels = mlb.fit_transform(df[label].dropna())
  print(list(mlb.classes_))
  mlb_df = pd.DataFrame(columns=mlb.classes_, data=labels)
  mlb_df.head()
  n_df = pd.concat([n_df,mlb_df],axis=1)
  n_df = n_df.dropna()

MLB('developer')
MLB('game_details')
MLB('languages')
MLB('genre')

# n_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/di

['', ' "Adiart"', ' "FieryElsa"', ' "Xblade"', ' & Jeya', ' & Windows Update)', ' Ben Palgi', ' Bianca Zankl', ' Bucharest', ' Carlo Castellano', ' China', ' Co.', ' Commander', ' Devendra Pohly', ' Diedemor studio', ' Empty Keys', ' Gabriel Mittermair', ' Gearbox Software', ' GmbH', ' INC.', ' Inc', ' Inc.', ' Jukio', ' Julian Heinken', ' Kaleb', ' Kiev', ' Kitty', ' L.L.C', ' LLC', ' LLC Self', ' LLC)', ' LLC.', ' LTD', ' LTD.', ' Limited', ' Linux', ' Lion Game Lion', ' Ltd', ' Ltd.', ' Massive Entertainment', ' Montpellier', ' Montreal', ' Nexon Korea Corporation', ' Otto Hantula', ' PC Port - Hardlight', ' PerspectX', ' Phantasm Games', ' Pixel Dash Studios', ' Reactor', ' Red Storm', ' S.C.', ' S.L.', ' SL', ' Samantha Povolny', ' Sara Lisa Vogl', ' Shanghai', ' Singapore', ' Sofia', ' Sun Pixel', ' Tobias Offermanns', ' Tom Jubert', ' Toronto', ' Toronto studios', ' Ubisoft Kiev', ' You Shut Up Inc.', ' a Ubisoft Studio', ' a.s.', ' and Dom', ' and Ubisoft Shanghai', ' in collab

# 로지스틱 1 

In [ ]:
X = n_df.drop(['all_reviews'], axis=1) 
y = n_df['all_reviews']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=100)

model = LogisticRegression(C=1,penalty = 'l2')
model.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
# 로지스틱 모델 학습 성능 비교
y_pred = model.predict(X_test) # 예측 결과 라벨

# 정확도 측정
accuracy_score(y_pred, y_test) 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.051113360323886636

# 로지스틱 2

In [ ]:
from numpy import nan

#  연습장

In [ ]:
mlb =  MultiLabelBinarizer()

X = df['game_details'].dropna()
print(X)
a = mlb.fit_transform(X)
# print(a)
# X = [['A','D'],['C','D','E'],['B','D','G'],['A','D','B'],['B','D','F'],['A','D','G'],['C','D','E']]
# print(df['game_details'].tolist())
# print(df['game_details'])
# mlb.fit_transform([['Single-player', 'Multi-player', 'Co-op', 'Steam Achievements', 'Steam Trading Cards', 'Partial Controller Support', 'Steam Cloud'], ['Multi-player', 'Online Multi-Player', 'Stats'], ['Single-player', 'Multi-player', 'Online Multi-Player', 'Cross-Platform Multiplayer', 'Steam Achievements', 'Steam Trading Cards', 'Steam Cloud'], ['Multi-player', 'Online Multi-Player', 'Steam Workshop', 'Steam Cloud', 'Valve Anti-Cheat enabled']])
# mlb.fit_transform(X)


# #create boolean mask matched non NaNs values
# mask = df['game_details'].notnull()

# #filter by boolean indexing
# arr = mlb.fit_transform(df.loc[mask, 'game_details'].dropna().str.strip('[]').str.split(','))

# #create DataFrame and add missing (NaN)s index values
# df = (pd.DataFrame(arr, index=df.index[mask], columns=mlb.classes_)
#                .reindex(df.index, fill_value=0))

0        Single-player,Multi-player,Co-op,Steam Achieve...
1                   Multi-player,Online Multi-Player,Stats
2        Single-player,Multi-player,Online Multi-Player...
3        Multi-player,Online Multi-Player,Steam Worksho...
4        Multi-player,Online Multi-Player,MMO,Co-op,Onl...
                               ...                        
40557    Single-player,Multi-player,Online Multi-Player...
40598    Single-player,Multi-player,Online Multi-Player...
40611    Single-player,Steam Achievements,Steam Trading...
40728    Single-player,Steam Achievements,Profile Featu...
40811    Single-player,Multi-player,Online Multi-Player...
Name: game_details, Length: 17001, dtype: object


# 출력

In [ ]:
n_df.head()


,all_reviews,,"""Adiart""","""FieryElsa""","""Xblade""",& Jeya,& Windows Update),Ben Palgi,Bianca Zankl,Bucharest,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,1,0,0,0
